In [1]:
from dotenv import load_dotenv
import os
import json
load_dotenv(override=True)

import time
from datetime import datetime 



In [2]:
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")

In [3]:
# connect to db
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user=USER,
  password=PASSWORD,
  database="mpp21"
)

mycursor = mydb.cursor()

In [4]:
WATSON_KEY = "4NYqVYmsjQ7gOWjO6rO-JxDVuyt_eac-5JPgqsQlC3w3"
WATSON_URL = "https://gateway-fra.watsonplatform.net/tone-analyzer/api"

In [5]:
from ibm_watson import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    iam_apikey=WATSON_KEY,
    url=WATSON_URL
)

In [93]:
def select_articles(curs, db, n):
    
    curs.execute("""SELECT article_id, title, content 
                FROM articles 
                WHERE newspaper = '""" + n + """' 
                AND is_privacy = 'privacy'
                """) #AND tone = ""
    
    arts = {}
    result = curs.fetchall()
    for r in result:
        arts[r[0]] = {
            "content": r[1] + " " + r[2]
            
        }
        if r[0] == "TDP_1917":
                print(r[1])
    
    return arts

In [7]:
def get_tone(analyzer, content):
    try:
        tone_analysis = analyzer.tone(
            {'text': content},
            content_type='application/json'    
        ).get_result()
    except:
        print("err")
        tone_analysis = "error"
    
    return tone_analysis

In [8]:
def update_tone(curs, db, a, tone_result, tone):

    insertQuery = "UPDATE articles SET tone_result = (%s), tone = (%s) WHERE article_id = (%s)"
    insertValues = (tone_result, tone, a)
    
    curs.execute(insertQuery, insertValues)
    
    db.commit() 

In [94]:
articles = select_articles(mycursor, mydb, "TDP")
len(articles)

Chipping at freedom


1227

In [95]:
articles["TDP_1917"]["content"]

"Chipping at freedom Do I let my boss microchip me (Aug 2) raises some interesting issues. but there are huge risks.Yes, it will mean no more riding the train or bus for free, or making up an excuse for why we got to work late as machines monitors where we are and when we arrive. Like payWave as we pass by a sensor, payments are automatically deducted.However, with this, service providers will have more access to more information, which will also severely limit our freedoms. For instance, will we still be able to choose to pay with cash or our credit card or be forced to pay with chip? The information on our chips could be corrupted, wiped or copied. It would make it easy for Big Brother to continuously track where we are, what we do, how we do it and with whom.Then there are potential health concerns. One review of academic literature found that in eight of 11 studies of microchipping of animals, ''in almost all cases, the malignant tumours, typically sarcomas, arose at the site of th

In [96]:
# get sample of tone
sample_tone = get_tone(tone_analyzer, articles["TDP_1917"]["content"])#["document_tone"]["tones"]
#print(sample_tone["sentences_tone"])#["tones"])

In [97]:
for i in sample_tone["sentences_tone"]:
    print(i)
    print()

{'tones': [{'tone_name': 'Joy', 'score': 0.577016, 'tone_id': 'joy'}, {'tone_name': 'Tentative', 'score': 0.615352, 'tone_id': 'tentative'}, {'tone_name': 'Analytical', 'score': 0.562568, 'tone_id': 'analytical'}], 'sentence_id': 0, 'text': 'Chipping at freedom Do I let my boss microchip me (Aug 2) raises some interesting issues.'}

{'tones': [], 'sentence_id': 1, 'text': 'but there are huge risks.Yes, it will mean no more riding the train or bus for free, or making up an excuse for why we got to work late as machines monitors where we are and when we arrive.'}

{'tones': [{'tone_name': 'Analytical', 'score': 0.676008, 'tone_id': 'analytical'}], 'sentence_id': 2, 'text': 'Like payWave as we pass by a sensor, payments are automatically deducted.However, with this, service providers will have more access to more information, which will also severely limit our freedoms.'}

{'tones': [{'tone_name': 'Tentative', 'score': 0.769251, 'tone_id': 'tentative'}, {'tone_name': 'Analytical', 'score'

In [39]:
all_tone_results = {}
all_tones = {}

ctr = 0

error_articles = []

start_time = datetime.now() 
print("Article processing started at: ", start_time)


for a in articles.keys(): # for each article
    

    
    returned_tones = get_tone(tone_analyzer, articles[a]["content"])#["document_tone"]["tones"]

    if returned_tones == "error":
        returned_tones = json.dumps({"error": "error"})
        tone = json.dumps({"error": "error"})
        error_articles.append(a)
        print()
        print(a)
        print()

    else:
        tone_result = json.dumps(returned_tones["document_tone"]["tones"])

        tone = {}

        for t in returned_tones["document_tone"]["tones"]:
            tone[t["tone_id"]] = t["score"]

        tone = json.dumps(tone)
    
#     except:
#         returned_tones = json.dumps({"error"})
#         tone = json.dumps({"error"})
#         error_articles.append(a)
#         print()
#         print(a)
#         print()
    
    
    all_tone_results[a] = tone_result
    all_tones[a] = tone
    
    ctr += 1
    if ctr % 100 == 0:
        time_elapsed = datetime.now() - start_time 
        print(str(ctr), " articles done. Time elapsed (hh:mm:ss.ms) {}".format(time_elapsed))
        
time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Article processing started at:  2021-03-06 13:08:01.229389
err

TG_186

100  articles done. Time elapsed (hh:mm:ss.ms) 0:01:49.736479
200  articles done. Time elapsed (hh:mm:ss.ms) 0:03:46.625698
300  articles done. Time elapsed (hh:mm:ss.ms) 0:05:43.839245
400  articles done. Time elapsed (hh:mm:ss.ms) 0:07:40.761788
500  articles done. Time elapsed (hh:mm:ss.ms) 0:09:33.347721
600  articles done. Time elapsed (hh:mm:ss.ms) 0:11:18.884929
700  articles done. Time elapsed (hh:mm:ss.ms) 0:13:06.880087
800  articles done. Time elapsed (hh:mm:ss.ms) 0:15:06.767361
900  articles done. Time elapsed (hh:mm:ss.ms) 0:17:03.839902
1000  articles done. Time elapsed (hh:mm:ss.ms) 0:18:57.364417
1100  articles done. Time elapsed (hh:mm:ss.ms) 0:20:55.332785
1200  articles done. Time elapsed (hh:mm:ss.ms) 0:22:48.692807
1300  articles done. Time elapsed (hh:mm:ss.ms) 0:24:43.056399
1400  articles done. Time elapsed (hh:mm:ss.ms) 0:26:35.321437
1500  articles done. Time elapsed (hh:mm:ss.ms) 0:28:27

In [40]:
print(len(all_tone_results))
print(len(all_tones))

5805
5805


In [41]:
start_time = datetime.now() 
print("Article processing started at: ", start_time)

for a in articles.keys(): 
    try:
        update_tone(mycursor, mydb, a, all_tone_results[a], all_tones[a])
    except:
        pass


time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Article processing started at:  2021-03-06 15:00:14.647038
Time elapsed (hh:mm:ss.ms) 0:00:47.407279


In [42]:
error_articles

['TG_186', 'TG_2649']

In [58]:
def select_articles_notone(curs, db):
    
    curs.execute("""SELECT article_id, tone
                FROM articles 
                WHERE is_privacy = 'privacy'
                """)
    
    arts = {}
    result = curs.fetchall()
    for r in result:
        arts[r[0]] = {
            "tone": r[1]
        }
    
    return arts

In [88]:
temp = select_articles_notone(mycursor, mydb)
len(temp)

26706

In [89]:
for i in temp.keys():
    if "error" in json.loads(temp[i]["tone"]).keys():
        print(i)

In [93]:
len(articles["TG_186"]["content"])#[:128000]) by 15727 bytes

143727

In [84]:
# returned_tones = get_tone(tone_analyzer, articles["TG_186"]["content"][:128000])#["document_tone"]["tones"]


In [86]:
# tone_result = json.dumps(returned_tones["document_tone"]["tones"])

# tone = {}

# for t in returned_tones["document_tone"]["tones"]:
#     tone[t["tone_id"]] = t["score"]

# tone = json.dumps(tone)

In [87]:
# update_tone(mycursor, mydb, "TG_186", tone_result, tone)
